# Case Study 1: Natural vs Inner Joins

Two types of joins:
1. `Natural Join`  
    A Natural Join is where 2 tables are joined on the basis of all common columns.      
    ie. `left.join(right, 'key')`

2. `Inner Join`  
    A Inner Join is where 2 tables are joined on the basis of common columns mentioned in the ON clause.
    ie. `left.join(right, left[lkey] == right[rkey])`

source: https://stackoverflow.com/a/8696402

**Question:**
    Which is better? Is it just a style choice?

### Library Imports

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

Create a `SparkSession`. No need to create `SparkContext` as you automatically get it as part of the `SparkSession`.

In [2]:
spark = SparkSession.builder \
    .master("local") \
    .appName("Exploring Joins") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

sc = spark.sparkContext

### Initial Datasets

In [3]:
df_1 = spark.createDataFrame(
    [
        (1, 1, 'a'), 
        (2, 1, 'b'), 
        (2, 2, 'c'), 
    ], ['id', 'data_id', 'val_1']
)

df_1.toPandas()

,id,data_id,val_1
0,1,1,a
1,2,1,b
2,2,2,c


In [4]:
df_2 = spark.createDataFrame(
    [
        (1, 1, 10), 
        (2, 2, 20), 
    ], ['shop_id', 'data_id', 'val_2']
)

df_2.toPandas()

,shop_id,data_id,val_2
0,1,1,10
1,2,2,20


## Option 1: Natural Join

In [5]:
df_3 = df_1.withColumnRenamed('id', 'shop_id')

df = df_3.join(df_2, 'shop_id')

df.toPandas()

,shop_id,data_id,val_1,data_id,val_2
0,1,1,a,1,10
1,2,1,b,2,20
2,2,2,c,2,20


In [6]:
df.explain()

== Physical Plan ==
*(5) Project [shop_id#12L, data_id#1L, val_1#2, data_id#7L, val_2#8L]
+- *(5) SortMergeJoin [shop_id#12L], [shop_id#6L], Inner
   :- *(2) Sort [shop_id#12L ASC NULLS FIRST], false, 0
   :  +- Exchange hashpartitioning(shop_id#12L, 200)
   :     +- *(1) Project [id#0L AS shop_id#12L, data_id#1L, val_1#2]
   :        +- *(1) Filter isnotnull(id#0L)
   :           +- Scan ExistingRDD[id#0L,data_id#1L,val_1#2]
   +- *(4) Sort [shop_id#6L ASC NULLS FIRST], false, 0
      +- Exchange hashpartitioning(shop_id#6L, 200)
         +- *(3) Filter isnotnull(shop_id#6L)
            +- Scan ExistingRDD[shop_id#6L,data_id#7L,val_2#8L]


**What Happened**:
* An extra `Project` was performed before the join due to the `withColumnRenamed`.
* The `id` column disappeared in the result.

## Option 2: Regular Join

In [7]:
join_condition = df_1['id'] == df_2['shop_id']

df = df_1.join(df_2, join_condition)

df.toPandas()

,id,data_id,val_1,shop_id,data_id,val_2
0,1,1,a,1,1,10
1,2,1,b,2,2,20
2,2,2,c,2,2,20


In [8]:
df.explain()

== Physical Plan ==
*(5) SortMergeJoin [id#0L], [shop_id#6L], Inner
:- *(2) Sort [id#0L ASC NULLS FIRST], false, 0
:  +- Exchange hashpartitioning(id#0L, 200)
:     +- *(1) Filter isnotnull(id#0L)
:        +- Scan ExistingRDD[id#0L,data_id#1L,val_1#2]
+- *(4) Sort [shop_id#6L ASC NULLS FIRST], false, 0
   +- Exchange hashpartitioning(shop_id#6L, 200)
      +- *(3) Filter isnotnull(shop_id#6L)
         +- Scan ExistingRDD[shop_id#6L,data_id#7L,val_2#8L]


**What Happened**:
* No `Project` was done before the join.
* Both join keys are left in the result.

## Option 3: Inner Join, Same Column

In [9]:
join_condition = df_1['data_id'] == df_2['data_id']

df = df_1.join(df_2, join_condition)

df.toPandas()

,id,data_id,val_1,shop_id,data_id,val_2
0,1,1,a,1,1,10
1,2,1,b,1,1,10
2,2,2,c,2,2,20


In [10]:
df.explain()

== Physical Plan ==
*(5) SortMergeJoin [data_id#1L], [data_id#7L], Inner
:- *(2) Sort [data_id#1L ASC NULLS FIRST], false, 0
:  +- Exchange hashpartitioning(data_id#1L, 200)
:     +- *(1) Filter isnotnull(data_id#1L)
:        +- Scan ExistingRDD[id#0L,data_id#1L,val_1#2]
+- *(4) Sort [data_id#7L ASC NULLS FIRST], false, 0
   +- Exchange hashpartitioning(data_id#7L, 200)
      +- *(3) Filter isnotnull(data_id#7L)
         +- Scan ExistingRDD[shop_id#6L,data_id#7L,val_2#8L]


**What Happened**:
* No `Project` was done before the join.
* Duplicate columns appear in the result.

## TL;DR

**Natural Join** (Better)
* An extra `Project [id#0L AS shop_id#12L, data_id#1L, val_1#2]` was done, slightly less performant.
* Only a single column for the key appeared in the result.

**Inner Join**
* Both join keys appeared in the result.


## All to Say

I personally perfer doing a `natural join` because:
* The **join clause is a lot more elegant**, especially when it comes to joining on multiple columns.

* The resultant df **consists of only 1 join key**, the second key usually gets dropped afterwards in a `inner join` **.

**NOTE:** These rules also apply to the other join types (ie. `left` and `right`).

**Some might argue that you will need both join keys in the result for further transformations such as filter only the left or right key, but I would recommend doing this before the join, as this is more performant.